In [4]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import sys, os, pickle
import scipy.sparse
sys.path.append('/home/thijs/repos/dnp-code/PGM3_correct/source/')
from fishualizer_utilities import Zecording  # custom data loading class


In [2]:
base_path = '/home/thijs/Desktop/zf_rbm_essentials/'  # directory of h5 file with data
data_sets = {'20180912-Run01': '20180912_Run01_spontaneous_rbm2.h5'}  # fish name & file name 
test_segs = '267'  # which train/test split to use => we use 267 in the manuscript. 
train_inds_path=f'/home/thijs/repos/dnp-code/train_test_inds/20180912-Run01/train_test_inds__test_segs_{test_segs}_nseg10.pkl'   # path & file name of train/test split file


In [5]:
## Load data
recordings = {}  # I'm using a dictionary structure here, but you can ignore that 
for data_set, data_path in data_sets.items():  # load data via fishualizer Zecording object
    recordings[data_set] = Zecording(path=base_path + data_path, kwargs={'ignorelags': True,
                                              'forceinterpolation': False,
                                              'ignoreunknowndata': True,
                                              'loadram': True})  
rec = recordings[list(data_sets.keys())[0]]
print(rec)
regions = {'wb': np.arange(294)} # select all zbrain atlas regions. (again, a dictionary structure that you don't need)
selected_neurons = {}
n_sel_cells = {}
train_data = {}
test_data = {}
full_data = {}

dict_tt_inds = pickle.load(open(train_inds_path, 'rb'))  # load dictionary with training indices
train_inds = dict_tt_inds['train_inds']  # load training indices
test_inds = dict_tt_inds['test_inds']  # load test indices
print(f'Length of test inds {len(test_inds)}')
for ir in list(regions.keys()):
    selected_neurons[ir] = np.unique(scipy.sparse.find(rec.labels[:, regions[ir]])[0])  # find neurons that are connected to regions
    assert rec.spikes.shape[0] > rec.spikes.shape[1]  # double check orientation
    train_data[ir] = rec.spikes[selected_neurons[ir], :][:, train_inds]  # train data
    test_data[ir] = rec.spikes[selected_neurons[ir], :][:, test_inds]  # test data
    n_sel_cells[ir] = len(selected_neurons[ir])
    full_data[ir] = rec.spikes[selected_neurons[ir], :]  # all data

len test inds 1665
